In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [53]:
file = pd.read_csv('../feets_full_table.csv', sep=';')
file.head(14)

,id,sector,Amplitude,Eta_e,LinearTrend,Mean,Meanvariance,MedianAbsDev,PercentAmplitude,PeriodLS,Period_fit,Psi_CS,Psi_eta,Std,Class
0,1152358,22,0.038367,2.781633,8.398356e-06,0.999896,0.019490,0.013162,0.067496,0.000091,1.000000,0.013116,2.013830,0.019488,Noisy
1,4289780,6,0.003984,0.027998,-2.574445e-06,1.000369,0.002672,0.002411,0.006699,0.853793,0.000000,0.230237,0.050262,0.002673,Modul
2,4289780,32,0.004005,0.026885,3.039774e-06,1.000378,0.002702,0.002441,0.006885,0.854841,0.000000,0.233032,0.037314,0.002703,Modul
3,4373827,20,0.065865,2.627467,-1.699160e-05,0.999851,0.033532,0.022580,0.117115,0.000043,1.000000,0.019476,1.983205,0.033527,Noisy
4,4373827,44,0.070649,2.481889,3.691981e-05,1.000068,0.035810,0.024006,0.123233,0.000106,1.000000,0.015542,2.024841,0.035813,Noisy
5,4373827,45,0.062061,2.524782,-1.545700e-05,1.000097,0.031706,0.021571,0.109070,0.000040,0.999997,0.016329,1.981143,0.031709,Noisy
6,4373827,46,0.060725,2.504261,2.451540e-06,1.000440,0.031051,0.020825,0.108453,0.000040,1.000000,0.013413,1.995121,0.031064,Noisy
7,4373827,47,0.068740,2.602666,2.811777e-06,0.999550,0.035040,0.023848,0.122027,0.000036,1.000000,0.014799,1.978180,0.035024,Noisy
8,4373827,60,0.071001,3.407248,-6.563091e-05,1.000195,0.035988,0.024467,0.125629,0.000107,0.000000,0.023045,1.994281,0.035995,Noisy
9,4491131,20,0.068080,0.857899,-6.655137e-07,1.001776,0.037168,0.028319,0.125210,0.230257,0.000000,0.187549,0.665165,0.037234,Modul


In [42]:
group = file.groupby('Class')
group.count()

,id,sector,Amplitude,Eta_e,LinearTrend,Mean,Meanvariance,MedianAbsDev,PercentAmplitude,PeriodLS,Period_fit,Psi_CS,Psi_eta,Std
Class,,,,,,,,,,,,,,
Modul,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1113,1115,1115,1115
Noisy,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1713,1717,1717,1717


In [60]:
file['Class'][13][-1]

'l'

## Predictors and Class

In [29]:
x_feets = file.iloc[:, 2:14].values
y_feets = file.iloc[:, 14].values
#y_feets = y_feets.reshape(-1, 1)

In [30]:
print(x_feets.shape, type(x_feets))

(3202, 12) <class 'numpy.ndarray'>


In [31]:
print(y_feets.shape, type(y_feets))

(3202,) <class 'numpy.ndarray'>


In [32]:
y_feets

array(['Noisy', 'Modul', 'Modul', ..., 'Modul', 'Modul', 'Modul'],
      shape=(3202,), dtype=object)

## Attribute scaling

In [33]:
# Verifying infinity values

print(f'There is {np.isinf(x_feets).sum()} infinity values')

There is 5 infinity values


In [34]:
# Replacing inf values to nan and checking

x_feets = np.where(np.isinf(x_feets), np.nan, x_feets)

print(f'There is {np.isinf(x_feets).sum()} infinity values')

There is 0 infinity values


In [35]:
# Treating infinity values

imputer = SimpleImputer(strategy='median')
x_feets = imputer.fit_transform(x_feets)

In [36]:
scaler_feets = StandardScaler()

x_feets = scaler_feets.fit_transform(x_feets)

In [14]:
print(f'Valores máximos: {np.max(x_feets)}')
print(f'Valores mínimos: {np.min(x_feets)}')

Valores máximos: 43.94946935951369
Valores mínimos: -27.95905210699225


## LabelEncoder

In [45]:
encoder = LabelEncoder()
y_feets = encoder.fit_transform(y_feets)
y_feets

array([1, 0, 0, ..., 0, 0, 0], shape=(3202,))

In [49]:
print(encoder.classes_)
print(np.unique(y_feets))

[0 1 2]
[0 1 2]


## Training and test basis

In [38]:
x_feets_training, x_feets_test, y_feets_training, y_feets_test = train_test_split(x_feets, y_feets, test_size=0.25, random_state=0)

In [39]:
x_feets_training.shape, y_feets_training.shape

((2401, 12), (2401,))

In [40]:
x_feets_test.shape, y_feets_test.shape

((801, 12), (801,))

## Saving the data

In [41]:
with open('feets_data.pkl', mode='wb') as f:
    pickle.dump([x_feets_training, x_feets_test, y_feets_training, y_feets_test], f)